In [1]:
import numpy as np

# Part 1

Need to find invalid IDs for given input. An invalid ID is some sequence of digits repeated twice, in each given range

### Data Input and Conversion

In [2]:
# Import raw data as numpy array
data = np.loadtxt(
    fname='Input2.txt',
    delimiter=',',
    dtype='O'
)
# convert to 2 column ndarray
id_ranges = np.zeros(
    shape=(len(data),),
    dtype=[('start', int), ('end', int)]
    # dtype=int
)
# iterate through raw data to populate array
for i,j in enumerate(data):
    _id_range = j.split('-')
    id_ranges['start'][i] = _id_range[0]
    id_ranges['end'][i] = _id_range[1]

In [3]:
id_ranges

array([(   5529687,    5587329), (        50,         82),
       (       374,        560), (        83,        113),
       (    226375,     287485), (    293169,     368713),
       (      2034,       2634), (   9945560,    9993116),
       (   4872472,    4904227), (      3218,       5121),
       (      1074,       1357), (     15451,      26093),
       ( 483468003,  483498602), (     51513,      85385),
       (      1466,       1992), (      7600,      13034),
       (    710570,     789399), (    407363,     480868),
       (3996614725, 3996662113), (         3,         17),
       (5414907798, 5414992881), (     86274,     120443),
       (    828669,     909588), (    607353,     700604),
       (4242340614, 4242556443), (     28750,      44009),
       (    935177,    1004747), (        20,         41),
       (  74678832,   74818251), (8484825082, 8484860878),
       (2784096938, 2784156610), (      5477,       7589),
       (       621,        952), (2424167145, 2424278200

### Helpers

In [4]:
def digit_count(input_int: int) -> int:
    """For a given integer input, return number of digits."""
    # instantiate counter
    digit_counter = 0

    # increment digit counter using modulo function
    while True:
        if (input_int % (10**digit_counter)) == input_int:
            return digit_counter
        else:
            digit_counter += 1
        

In [5]:
def retrive_digits(number: int, position: int, count: int = 1) -> int:
    """Returns digit(s) of integer from given position, 
    
    1-indexed from units upwards.
    """
    # bound the position argument
    if (position > digit_count(number)) | (position < 1):
        return 0
    
    # modulo function logic for retrival of digits
    return int(
        ((number % 10**position) - (number % 10**(position-count))) / 10**(position-count)
    )

In [6]:
def invalids(start: int, end: int) -> list:
    """Given an integer range, produce a list of invalid IDs."""
    # instantiate output list of invalid IDs
    output_list = []
    # instantiate list of digit lengths to generate IDs from
    digit_lengths = []
    # instantiate working variable for ID generation
    current_id = 0

    # restrict domain to numbers with even number of digits
    for i in range(digit_count(start), digit_count(end)+1):
        if i % 2 == 0:
            digit_lengths.append(i)
    
    # for each set of digit lengths, set min / max of the domain
    for i in digit_lengths:
        domain_start = 10**(i-1) if digit_count(start) != i else start  # e.g. if start=3, floor at 10
        domain_end = 10**(i)-1 if digit_count(end) != i else end  # e.g. if end=1,000,000, cap at 999,999
        
        # start from beginning of domain
        current_id = domain_start
        
        # cut the start of range into halves
        half_one = retrive_digits(current_id, i, i/2)  # e.g. 935177 -> 935
        half_two = retrive_digits(current_id, i/2, i/2)  # e.g. 935177 -> 177
        
        # plan of attack: increment the start half 1 (h1) and replicate to h2
        # subject to range constraints
        while current_id <= domain_end:
            if half_one >= half_two:
                current_id = int(half_one*(10**(i/2)) + half_one)
                if current_id <= domain_end:
                    output_list.append(current_id)
            # increment 
            half_one += 1
            half_two = half_one
            # stitch the two halves back together...
            current_id = int(half_one*(10**(i/2)) + half_one)
            
    return output_list

### Process Ranges

In [7]:
def generate_invalid_ids(input_array: np.array) -> np.array:
    """For a given input array: test, generate and sum invalid IDs for each range."""
    
    # Generate a boolean mask where there are potential invalid IDs.
    # Premised on even number of digits in either start or end of range.
    mask_array = np.zeros(
        shape=(len(input_array),),
        dtype=bool
    )
    for i, j in enumerate(input_array):
        mask_array[i] = \
        (digit_count(j['start']) % 2 == 0) | (digit_count(j['end']) % 2 == 0)
    
    # filtered array
    filtered_array = input_array[mask_array]
    
    # instantiate output array
    output_array = np.zeros(
        shape=len(filtered_array,),
        dtype=object
    )
    # instantiate counter for invalid ID sum
    counter = 0
    
    # array of dicts: key = tuple of range, value = list of invalid IDs
    for i,j in enumerate(filtered_array):
        output_array[i] = {(int(j['start']), int(j['end'])): invalids(j['start'], j['end'])}
        
        counter += sum(invalids(j['start'], j['end']))
    
    print(f"sum of invalid IDs: {counter}")
    
    # array of ints
    return output_array

In [8]:
res = generate_invalid_ids(id_ranges)

sum of invalid IDs: 30599400849


# Part 2

Need to find new invalid IDs for given input. An invalid ID is some sequence of digits repeated AT LEAST twice, in each given range

### Updated Helpers

In [9]:
def stitch(numbers: list) -> int:
    """Given a list of single integers, stitch back to one integer."""
    # output integer
    output = 0
    
    # error check argument
    if (len(numbers) < 1):
        return 0

    # stitch together
    for i in range(len(numbers), 0, -1):
        output += (10**(i-1)) * numbers[len(numbers) - i]
    return int(output)

In [10]:
def invalids_v2(start: int, end: int) -> list:
    """Given an integer range, produce a list of invalid IDs.
    
    Any sequence repeated at least twice e.g. 998-1012 has two invalid IDs, 999 and 1010.
    """
    # instantiate output list of invalid IDs
    output_list = []
    # instantiate list of digit lengths to generate IDs from
    digit_lengths = []
    # instantiate working variable for ID generation
    current_id = 0
    # instantiate list of equal length 'pieces' of each ID
    pieces = []
    
    # generate domain of digits lengths
    for i in range(digit_count(start), digit_count(end)+1):
        digit_lengths.append(i)
    
    # for each set of digit lengths (i), set min / max of the domain
    for i in digit_lengths:
        domain_start = 10**(i-1) if digit_count(start) != i else start  # e.g. if start=3, floor at 10
        domain_end = 10**(i)-1 if digit_count(end) != i else end  # e.g. if end=1,000,000, cap at 999,999
        
        # start from beginning of domain
        current_id = domain_start
        
        # cut the ID into between [2,i] pieces (j)
        for j in range(2, i+1):
            if i % j == 0:  # if evenly cut
                
                # digit size of each piece
                piece_size = int(i/j)
                # number of pieces... just j but helps to work in english
                num_pieces = int(j)
                
                for k in range(i): 
                    # break into single digits w/ iterator k (note: piece size = i/j still here!)
                    pieces.append(
                        retrive_digits(current_id, i - k)  # e.g. 998 -> [9, 9, 8]
                    )
                
                # plan of attack: increment the start piece (p1) and replicate to all others (px)
                # subject to range constraints
                while current_id <= domain_end:
                    
                    # if first piece is larger than next piece
                    if stitch(pieces[:piece_size]) >= stitch(pieces[piece_size:piece_size*2]):
                        
                        # make an invalid ID based on repetition of first piece
                        current_id = stitch(
                            [y for x, y in enumerate(pieces) if (x % piece_size) == x] * num_pieces
                        )
                        # add to output, given still in range
                        if (current_id >= domain_start) & (current_id <= domain_end):
                            output_list.append(current_id)
    
                    # else increment first piece
                    pieces[piece_size - 1] += 1
                    
                    # then make a new invalid ID...
                    current_id = stitch(
                            [y for x, y in enumerate(pieces) if (x % piece_size) == x] * num_pieces
                        )
                    
                    # then store updated ID in pieces again
                    pieces.clear()
                    for k in range(i): 
                        pieces.append(
                            retrive_digits(current_id, i - k)
                        )
            
            # start from beginning of domain
            current_id = domain_start
            # # re-initialise pieces for new piece size (j)
            pieces.clear()

        # re-initialise pieces for new digit length (i)
        pieces.clear()

    # return invalids
    return np.unique(output_list)

In [11]:
def generate_invalid_ids_v2(input_array: np.array) -> np.array:
    """For a given input array: test, generate and sum invalid IDs for each range."""   
    # instantiate output array
    output_array = np.zeros(
        shape=len(input_array,),
        dtype=object
    )
    # instantiate counter for invalid ID sum
    counter = 0
    
    # array of dicts: key = tuple of range, value = list of invalid IDs
    for i,j in enumerate(input_array):
        output_array[i] = {(int(j['start']), int(j['end'])): invalids_v2(j['start'], j['end'])}
        
        counter += sum(invalids_v2(j['start'], j['end']))
    
    print(f"sum of invalid IDs: {counter}")
    
    # array of ints
    return output_array

In [12]:
res = generate_invalid_ids_v2(id_ranges)

sum of invalid IDs: 46270373595
